# CVXPY + CVXPYgen + CFFI + Numba

In [1]:
import cffi
import cvxpy as cp
from cvxpygen import cpg
import numpy as np
import numba

from numba.core.typing import cffi_utils

# CVXPY

In [2]:
m, n = 3, 2
x = cp.Variable(n, name='x')
A = cp.Parameter((m, n), name='A', sparsity=[(0, 0), (0, 1), (1, 1)])
b = cp.Parameter(m, name='b')
problem = cp.Problem(cp.Minimize(cp.sum_squares(A @ x - b)), [x >= 0])

np.random.seed(0)
A.value = np.zeros((m, n))
A.value[0, 0] = np.random.randn()
A.value[0, 1] = np.random.randn()
A.value[1, 1] = np.random.randn()
b.value = np.random.randn(m)

problem.solve()

0.9550720544957328

# CVXPYgen

In [3]:
code_dir = 'code_dir'

In [4]:
cpg.generate_code(problem, code_dir=code_dir, solver='SCS', wrapper=True)

Generating code with CVXPYgen ...
CVXPYgen finished generating code.
Compiling python wrapper with CVXPYgen ... 
-- The C compiler identification is GNU 11.2.0
-- The CXX compiler identification is GNU 11.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Setting build type to 'Release' as none was specified.
-- Single precision floats (32bit) are OFF
-- Long integers (64bit) are OFF
-- COMPILER_OPTS = -DUSE_LAPACK -DCTRLC
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/work_folder/convex-plasticity/demo/cvxpygen/code_dir/c/build
Scanning dependencies of target cpg
[  0%] Building C object CMake

/mnt/work_folder/convex-plasticity/demo/cvxpygen/code_dir/c/solver_code/src/rw.c: In function ‘_scs_read_data’:
/mnt/work_folder/convex-plasticity/demo/cvxpygen/code_dir/c/solver_code/src/rw.c:184:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  184 |   fread(&(file_int_sz), sizeof(uint32_t), 1, fin);
      |   ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/mnt/work_folder/convex-plasticity/demo/cvxpygen/code_dir/c/solver_code/src/rw.c:185:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  185 |   fread(&(file_float_sz), sizeof(uint32_t), 1, fin);
      |   ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/mnt/work_folder/convex-plasticity/demo/cvxpygen/code_dir/c/solver_code/src/rw.c:202:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  202 |   fread(&(file_version_sz), sizeof(uint32_t), 1, fin);
      | 

[ 31%] Building C object CMakeFiles/cpg.dir/solver_code/src/scs_version.c.o
[ 34%] Building C object CMakeFiles/cpg.dir/solver_code/src/util.c.o
[ 37%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/csparse.c.o
[ 41%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/scs_matrix.c.o
[ 44%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/cpu/direct/private.c.o
[ 48%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/qdldl/qdldl.c.o
[ 51%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/SuiteSparse_config.c.o
[ 55%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_1.c.o
[ 58%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_2.c.o
[ 62%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_aat.c.o
[ 65%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_control.c.o
[ 68%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/a

In [5]:
# solve problem conventionally
val = problem.solve(solver='SCS')
val

0.9550720544957453

In [8]:
# import extension module and register custom CVXPY solve method
from code_dir.cpg_solver import cpg_solve
problem.register_solve('cpg', cpg_solve)

# solve problem with C code via python wrapper
val = problem.solve(method='cpg', updated_params=['A', 'b'], verbose=True)
val

0.9550542521966028

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3, constraints m: 7
cones: 	  l: linear vars: 2
	  q: soc vars: 5, qsize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 0, acceleration_interval: 0
lin-sys:  sparse-direct
	  nnz(A): 7, nnz(P): 0
WARN: aa_init returned NULL, no acceleration applied.
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.91e+01  1.00e+00  2.89e+01 -1.33e+01  1.00e-01  1.34e-04 
   125| 1.11e-05  9.42e-07  1.77e-05  9.55e-01  5.20e-01  2.25e-04 
----------

# CFFI

In [10]:
with open('cffi_wrapper/cffi_wrapper.c', 'r') as file:
    cdef = file.read()

with open('cffi_wrapper/cffi_wrapper.h', 'r') as file:
    source = file.read()
    
lib_dir = os.path.join(os.getcwd(), code_dir + '/c/build/out')
solver_include_dir = os.path.join(os.getcwd(), code_dir + '/c/solver_code/include')
include_dir = os.path.join(os.getcwd(), code_dir + '/c/include')

ffibuilder = cffi.FFI()

ffibuilder.set_source(
  module_name='_cpglib', 
  source=source,
  include_dirs = [include_dir, solver_include_dir],
  libraries = ['cpg'],
  library_dirs = [lib_dir],
)

ffibuilder.cdef(csource=cdef)

ffibuilder.compile()

'/mnt/work_folder/convex-plasticity/demo/cvxpygen/_cpglib.cpython-39-x86_64-linux-gnu.so'

# CFFI + Numba

In [11]:
# from _cpglib import ffi, lib
import _cpglib

cffi_utils.register_module(_cpglib)

CPG_Updated_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Updated_cpp_t'), use_record_dtype=True)
CPG_Params_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Params_cpp_t'), use_record_dtype=True)
CPG_Result_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Result_cpp_t'), use_record_dtype=True)

cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Updated_cpp_t'), CPG_Updated_python_t)
cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Params_cpp_t'), CPG_Params_python_t)
cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Result_cpp_t'), CPG_Result_python_t)

sig = cffi_utils.map_type(_cpglib.ffi.typeof(_cpglib.lib.solve_cpp), use_record_dtype=True)
# sig2 = types.void(types.CPointer(CPG_Updated_python_t), types.CPointer(CPG_Params_python_t), types.CPointer(CPG_Result_python_t))

@numba.cfunc(sig, nopython=True)
def solve_wrapper(upd, par, res):
    _cpglib.lib.solve_cpp(upd, par, res)

cpg_solver = solve_wrapper.ctypes

print(type(solve_wrapper), type(cpg_solver))

<class 'numba.core.ccallback.CFunc'> <class 'ctypes.CFUNCTYPE.<locals>.CFunctionType'>


In [23]:
def my_cpg_solve(problem: cp.problems.problem.Problem, **kwargs):
    
    #Preliminary initialization of parameters, functions, etc (before the numba-function invocation)

    upd = _cpglib.ffi.new("CPG_Updated_cpp_t *", {})
    par = _cpglib.ffi.new("CPG_Params_cpp_t *", {})
    res = _cpglib.ffi.new("CPG_Result_cpp_t *", {})

    upd_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(upd), dtype=numba.np.numpy_support.as_dtype(CPG_Updated_python_t), shape=1,)
    par_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(par), dtype=numba.np.numpy_support.as_dtype(CPG_Params_python_t), shape=1,)
    res_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(res), dtype=numba.np.numpy_support.as_dtype(CPG_Result_python_t), shape=1,)

    updated_params = ["A", "b"]
    for p in updated_params:
        setattr(upd, p, True)

    _cpglib.lib.cpg_set_solver_default_settings()

    for key, value in kwargs.items():
        try:
            eval(' _cpglib.lib.cpg_set_solver_%s(value)' % key)
        except AttributeError:
            raise(AttributeError('Solver setting "%s" not available.' % key))

    # set parameter values
    n = problem.param_dict['A'].shape[0]
    A_coordinates = np.unique([coord[0]+coord[1]*n for coord in problem.param_dict['A'].attributes['sparsity']])
    A_value = []
    A_flat = problem.param_dict['A'].value.flatten(order='F')
    for coord in A_coordinates:
        A_value.append(A_flat[coord])
        A_flat[coord] = 0

    A_value = np.array(A_value)
    b_value = np.array(problem.param_dict['b'].value.flatten(order='F'))

    @numba.njit
    def call_solver(upd_numpy: np.ndarray, par_numpy: np.ndarray, res_numpy: np.ndarray):
        
        #Do something inside the numba-function

        #Work with numpy arrays inside numba-function
        par_numpy['A'][:] = A_value
        par_numpy['b'][:] = b_value

        #Call c-type function generated by cvxpypgen
        cpg_solver(upd_numpy.ctypes.data, par_numpy.ctypes.data, res_numpy.ctypes.data)

        #Do something inside the numba-function

    call_solver(upd_numpy, par_numpy, res_numpy)

In [29]:
my_cpg_solve(problem, verbose=True, eps_abs=1e-13, eps_rel=1e-13)